In [1]:
import pandas as pd
import numpy as np
import sys
import json 
import random
import math
import collections
import seaborn as sns
import networkx as nx

## Load Data

In [8]:
G = nx.read_weighted_edgelist('../data/conn_communities.weighted.edgelist')
G = nx.relabel_nodes(G, {node:int(node) for node in G.nodes}) 
df = pd.read_csv('../data/conn_communities.csv',index_col=0)

df.head()

,index,position,name,city,menu,link,score,N_reviews,price,Name,...,specialDiets,positionlink,province,Link,provinceOK,louvain,kmeans,async_fluid_7,multicore,demon
0,0,2018,Osteria al Borgo,Lucca,False,/Restaurant_Review-g187898-d23137800-Reviews-O...,5.0,6,NaN,Osteria al Borgo,...,"Vegetarian Friendly, Vegan Options, Gluten Fre...",j39_https://maps.google.com/maps?saddr=&daddr=...,pisa,/Restaurant_Review-g187898-d23137800-Reviews-O...,0,8.0,1.0,6.0,2.0,58
1,12,1309,Il Genio Italiano,Florence,True,/Restaurant_Review-g187895-d2483611-Reviews-Il...,4.0,229,2.5,Il Genio Italiano,...,Vegetarian Friendly,vs1_https://maps.google.com/maps?saddr=&daddr=...,florence,/Restaurant_Review-g187895-d2483611-Reviews-Il...,1,8.0,0.0,6.0,2.0,58
2,37,1316,La Taverna,Scandicci,False,/Restaurant_Review-g644284-d2660933-Reviews-La...,4.0,236,2.5,La Taverna,...,Vegetarian Friendly,zhJ_https://maps.google.com/maps?saddr=&daddr=...,florence,/Restaurant_Review-g644284-d2660933-Reviews-La...,1,8.0,0.0,6.0,2.0,58
3,48,1301,Bar Pasticceria Deanna,Florence,False,/Restaurant_Review-g187895-d2077023-Reviews-Ba...,4.0,162,1.0,Bar Pasticceria Deanna,...,"Vegetarian Friendly, Vegan Options, Gluten Fre...",hSR_https://maps.google.com/maps?saddr=&daddr=...,florence,/Restaurant_Review-g187895-d2077023-Reviews-Ba...,1,8.0,0.0,6.0,3.0,58
4,50,1332,Perseus Fiesolano,Fiesole,True,/Restaurant_Review-g187896-d2720609-Reviews-Pe...,3.5,754,2.5,Perseus Fiesolano,...,Gluten Free Options,d5H_https://maps.google.com/maps?saddr=&daddr=...,florence,/Restaurant_Review-g187896-d2720609-Reviews-Pe...,1,8.0,0.0,6.0,3.0,58


In [23]:
dfg = pd.read_csv('../data/conn_communities.weighted.edgelist',index_col = None, header = None, sep=' ', names=['u','v','weight'])
dfg

,u,v,weight
0,0,1,1.0
1,0,2,1.0
2,0,3,1.0
3,0,4,1.0
4,0,5,1.0
...,...,...,...
136641,14067,14999,1.0
136642,14067,15000,2.0
136643,14999,15000,1.0
136644,14669,14892,1.0


In [17]:
dfg[dfg['weight'] == 0]

,u,v,weight


In [15]:
for node in list(G.nodes)[:10]: print(G.nodes[node])

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}


In [27]:
C = G.copy()
nodeList = list(C.nodes)

In [28]:
noedge = []

while len(noedge) < 136646:
    
    u = random.choice(nodeList)
    v = random.choice(nodeList)
    
    if u != v and C.has_edge(u,v) == False:
        
        C.add_edge(u,v, weight=0)
        dfg = dfg.append([u,v,0])
        noedge.append((u,v))

In [29]:
dfg.to_csv('../data/dfg.csv')

In [30]:
len(noedge)

136646

In [ ]:
dfg = pd.read_csv('../data/dfg.csv',index_col = 0)

In [ ]:
for i in dfg.index:
    
    u = dfg.loc[i,'u'] 
    v = dfg.loc[i,'v']
    
    if df.loc[u,'province'] == df.loc[v,'province']: dfg.loc[i,'province'] = 1
    else: dfg.loc[i,'province'] = 0
    if df.loc[u,'province'] == df.loc[v,'province']: dfg.loc[i,'province'] = 1
    else: dfg.loc[i,'province'] = 0
    
    xu = df.loc[u,'longitude']
    xv = df.loc[v,'longitude']
    yu = df.loc[u,'latitude']
    yv = df.loc[v,'latitude'] 
    dfg.loc[i,'geoDistance'] =  (((xu.xv)**2)  + ((yu-yv)**2))**0.5
    
    dfg.loc[i,'score'] = abs(df.loc[u,'score'] - df.loc[v,'score'])
    dfg.loc[i,'score'] = abs(df.loc[u,'score'] - df.loc[v,'score'])
    
    specialDietsu = set(df.loc[u,'specialDiets'].split(', '))
    specialDietsv = set(df.loc[v,'specialDiets'].split(', '))
    dfg.loc[i,'specialDiets'] = len(specialDietsu.interception(specialDietsv))
    
    specialDietsu = set(df.loc[u,'specialDiets'].split(', '))
    specialDietsv = set(df.loc[v,'specialDiets'].split(', '))
    dfg.loc[i,'specialDiets'] = len(specialDietsu.interception(specialDietsv))
    
    
    